# Proceso

- Importar librerias
- Carga de modelos
- Consulta datos diarios a synapse
- Limpieza y preprocesamiento de datos
- Configuración BETO
- Modelos:
  - Indicador pregunta
  - Sentimiento
  - Temas
  - Causa raiz
  - Palabra
  - Punto Interacción
  - Customer journey
- Carga de datos


- Resultado
Crea columnas en BT_COMENTARIOS
  - indicador_id, sentimiento,tema_id,Causa raiz_id,(descargar modelo),palabra,punto interacción_id,customer journey_id 

- Crea las siguientes tablas diccionarios 1 vez
  - LK_INDICADOR, LK_TEMA,LK_CAUSA_RAIZ,LK_PUNTO_INTERACCION,LK_CUSTOMER_JOURNEY

# Ejecuciones Unicas

In [ ]:
# Traer modelos del datalake para que los pueda llamar desde torch. esto se hace 1 sola vez!
# crear: dbutils.fs.cp('//NLP Models/model_CustomerJourneyV1_BETO_uncased_fine_tune.pkl','//model_CustomerJourneyV1_BETO_uncased_fine_tune.pkl.pkl')


# Instalar Librerias

In [ ]:
%pip install torch==1.9.0
%pip install click==8.0.4
%pip install spacy==3.1.3
%pip install transformers==4.15.0

#Importar Librerias

In [ ]:
import yaml #6.0'
import pandas as pd #1.2.4
import numpy as np #1.19.2
import torch #1.9.0
import json #2.0.9
import os
import re #'2.2.1
import spacy #3.1.3
import requests #2.25.1 
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pytz import timezone
from torch import nn #1.9.0+cpu 
from transformers import AutoTokenizer,AutoModelForCausalLM #4.15.0 
from transformers import BertModel,BertTokenizer #4.15.0 
from sklearn.model_selection import train_test_split #0.24.1 
from datetime import datetime, timedelta 

os.system('python -m spacy download es_core_news_lg')


#Param / Config

In [ ]:
# DEFINICION DE TIMEZONE
TIMEZONE_COLOMBIA = 'America/Bogota'

# PATH GENERACION LKs EN DATALAKE
path_contenedor_lk = '/fkt'
path_contenedor_bt = '/mnt'

# PARAMETROS DE LLAMADA
dbutils.widgets.text("param_FechaDesde", "", "Fecha Desde")
dbutils.widgets.text("param_FechaHasta", "", "Fecha Hasta")

# LEO ARCHICO YAML PARA RECUPERAR PARAMETROS DE CONEXION
dbutils.fs.cp('/s_secrets.yml', '/w_secrets.yml')
secret_file = "l_secrets.yml"

with open(secret_file, 'r') as f:
    try:
        yaml_args = yaml.safe_load(f)
        
        #Synapse
        synapse_url = yaml_args['connection']['sql_synapse']['url']
        synapse_user = yaml_args['connection']['sql_synapse']['user']
        synapse_password = yaml_args['connection']['sql_synapse']['password']
        synapse_host = yaml_args['connection']['sql_synapse']['host']
        synapse_database = yaml_args['connection']['sql_synapse']['databaseName']
                    
    except yaml.YAMLError:
        print('Error al parsear el yaml')
        raise


var_FechaDesde = dbutils.widgets.get("param_FechaDesde")
var_FechaHasta = dbutils.widgets.get("param_FechaHasta")

if not(var_FechaHasta): 
    var_FechaHasta_Query = datetime.now().astimezone(timezone(TIMEZONE_COLOMBIA)).strftime('%Y-%m-%d')
    var_FechaHasta = (datetime.now().astimezone(timezone(TIMEZONE_COLOMBIA)) + timedelta(days = -1)).strftime('%Y-%m-%d')
else:
    var_FechaHasta_Query = (datetime.strptime(var_FechaHasta, '%Y-%m-%d') + timedelta(days = 1)).strftime('%Y-%m-%d')

    
if not(var_FechaDesde): var_FechaDesde = (datetime.now().astimezone(timezone(TIMEZONE_COLOMBIA)) + timedelta(days = -1)).strftime('%Y-%m-%d')

print(f'Fecha Desde: {var_FechaDesde}')
print(f'Fecha Hasta: {var_FechaHasta}')
print(f'Fecha Hasta Query: {var_FechaHasta_Query}')

#Dataset Query

In [ ]:
# PI.INDICADOR, 
# JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID = I.INTERACCION_ID 
# LEFT JOIN EFECTY_VOC.LK_PREG_INDIC PI ON PI.PREGUNTA_ID = C.PREGUNTA_ID

query = f"""
        SELECT
        C.ID, D, I.MEDIO_INGRESO_ID, M.CANAL, M.MEDIO_INGRESO, W.WRAPUP, LS.SUBTIPO, P.INDICADOR, LKC.TIPO_CLIENTE 
        FROM EFECTY_VOC.BT_COMENTARIOS C 
        JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID = I.INTERACCION_ID 
        LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID = I.MEDIO_INGRESO_ID 
        WHERE 1=1 
        AND I.FECHA >= '{var_FechaDesde}' 
        AND I.FECHA < '{var_FechaHasta_Query}' 
        """


BT_COMENTARIOS = spark.read \
                      .format("jdbc") \
                      .option("url", synapse_url) \
                      .option("user", synapse_user) \
                      .option("password", synapse_password)\
                      .option("query", query) \
                      .load() \
                      .toPandas()


if len(BT_COMENTARIOS) <= 0:
    dbutils.notebook.exit('No se encontaron datos') #Aca termina la corrida porque no hay datos
else:
    print(f'Se recuperaron {len(BT_COMENTARIOS)} registros')

#Analisis Dataset Recuperado

In [ ]:
# ANALISIS 
BT_COMENTARIOS[['INTERACCION_ID','MEDIO_INGRESO','CANAL']].groupby(['CANAL','MEDIO_INGRESO']).count().sort_values(by='CANAL')

In [ ]:
#################################################### TRABAJO SOBRE EL TEXTO #######################
# BT_COMENTARIOS=BT_inicial.copy()
listl=[]
for i in BT_COMENTARIOS.columns.tolist():
    listl.append(i.lower())
dict_lower=dict(zip(list(BT_COMENTARIOS.columns),listl))
BT_COMENTARIOS=BT_COMENTARIOS.rename(columns=dict_lower)


if len(BT_COMENTARIOS)==0:
    print('No se encontraron datos que analizar')

#El comentario tal cual lo descargo, quedará con la marca "comentario inicial"
BT_COMENTARIOS['comentario_inicial']=BT_COMENTARIOS['comentario']

#Lo paso a string y lo hago todo lower
BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario'].astype(str)
index_ini=list(BT_COMENTARIOS.index)

BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x:x.lower())

#Modifico algunas cuestiones que afectan sobre todo a los PQRS: quito los /n, y los numeros a un mismo formato
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r'[\n]{1,4}',' ',x)) #Mas que nada para las PQRS que tienen muchas lineas vacias, donde hay mucho \n textual
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r"""[' ']{1,4}""",' ',x)) #CONTINUACIÓN DE LO DE ARRIBA
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r"""[0-9]{1,10}""",'numero',x))
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(' app|$app',' aplicación ',str(x)))

#Limpio comentarios de RRSS y Customer Service
index_rrss=BT_COMENTARIOS[(BT_COMENTARIOS['canal']=='RRSS')|(BT_COMENTARIOS['canal']=='CUSTOMER SERVICE')].index

BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.findall(".liente:(.*)\**",x)if 'liente:' in x else x)
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r'\?','zzz',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r'\.','bbb',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(',','qwqz',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub('[\W]',' ',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub('qwqz',',',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub('bbb','.',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub('zzz','?',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r'\s\s',' ',str(x)))
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub('Imagen .*? [0-9-a-z]{10,30}','',str(x)))


#Definicion BETO

In [ ]:
######################## DEFINICION DE BETO/NLP  #########################
#cargo el tokenizador de beto
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

#donde voy a trabajar
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
#Vuelvo a definir la estructura del modelo donde voy a trabajar ( es la misma que en el entrenamiento)
class BERTSentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTSentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("pytorch/")
        self.drop = nn.Dropout(p=0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, cls_output = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            return_dict=False
        )
        drop_output = self.drop(cls_output)
        output = self.linear(drop_output)
        return output

#Defino el proceso del modelo para analizar el texto
def classifySentiment(review_text):
    encoding_review = tokenizer.encode_plus(
        review_text,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        padding=True,
    #       pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )

    input_ids = encoding_review['input_ids'].to(device)
    attention_mask = encoding_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1) #El output es un vector de NCLASSES dimensiones (3). Aca, se va a quedar con el valor mas alto, quien será el elegido 
    #   return prediction.tolist()[0],output.tolist()[0][0],output.tolist()[0][1],output.tolist()[0][2]
    return prediction.tolist()[0]


#Pregunta

In [ ]:
######################## GENERACIÓN DEL INDICADOR: EL IDENTIFICACIÓN DE PREGUNTA #########################
#CLASIFICA EN NPS, SENTIMIENTO, ESFUERZO, NULL SATISFACCIÓN

#Selecciono el segmento al que quiero que corra el modelo
index_indicador=BT_COMENTARIOS[BT_COMENTARIOS['canal']=='ENCUESTA'].index

#Cargo el modelo
model = torch.load('/dbfs/Efecty_VoC/BETOmodel/modelV1_preguntas_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))
#Asi se entrenó a modelV1_preguntas_BETO_uncased_fine_tune. no se pueden modificar

#Genero el diccionario
# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_preg={0:'Null',1:'Satisfacción',2:'Esfuerzo',3:'NPS',4:'Sentimiento'}
query = "SELECT INDICADOR_ID, INDICADOR FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_LK_INDICADOR"
dict_preg = spark.read \
                 .format("jdbc") \
                 .option("url", synapse_url) \
                 .option("user", synapse_user) \
                 .option("password", synapse_password)\
                 .option("query", query) \
                 .load() \
                 .orderBy('INDICADOR_ID') \
                 .toPandas() \
                 .set_index('INDICADOR_ID') \
                 .T \
                 .to_dict('index')['INDICADOR']


BT_COMENTARIOS['indicador_id']=-99
BT_COMENTARIOS.loc[index_indicador,'indicador_id']=BT_COMENTARIOS.loc[index_indicador,'pregunta'].apply(lambda x:classifySentiment(x))



#Genero la tabla diccionario
schema = StructType([
    StructField('INDICADOR_ID', IntegerType(), False),
    StructField('INDICADOR', StringType(), False)
])

LK_INDICADOR = spark.createDataFrame(dict_preg.items(), schema) 
nombre_archivo = f'{path_contenedor_lk}/LK_INDICADOR'
LK_INDICADOR.write.mode('overwrite').parquet(nombre_archivo)

#Sentimiento

In [ ]:
############################# GENERACIÓN DEL SENTIMIENTO ##############################################

#Selecciono el segmento al que quiero que corra el modelo
index_sen=BT_COMENTARIOS[
                ((BT_COMENTARIOS['canal']=='PQRS')&(
                            (BT_COMENTARIOS['medio_ingreso']=='CORPORATIVO')|
                            (BT_COMENTARIOS['medio_ingreso']=='WEB')|
                            (BT_COMENTARIOS['medio_ingreso']=='CORREO ELECTRONICO')))|
                ((BT_COMENTARIOS['canal']=='RRSS'))|
                ((BT_COMENTARIOS['canal']=='ENCUESTA')&
                            (BT_COMENTARIOS['medio_ingreso']=='CRM')&
                            (BT_COMENTARIOS['indicador']==4))].index

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/BETO_sentimiento_V2.pkl', map_location=torch.device('cpu'))
#modelV9_BETO_uncased_fine_tune modelo viejo 19.4.2022

BT_COMENTARIOS.loc[index_sen,'sentimientoV4']=BT_COMENTARIOS.loc[index_sen,'comentario'].apply(lambda x:classifySentiment(x))

#Cargo el modelo
# https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased

#Trabajo sobre los resultados
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimientoV4'].apply(lambda x: 0.5 if x==1 else x)
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimiento'].apply(lambda x: 1 if x==2 else x)
BT_COMENTARIOS.drop(columns=['sentimientoV4'],inplace=True)

#Modifico el formato
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimiento'].apply(lambda x: -99 if pd.isna(x) else x)
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimiento'].apply(lambda x: float(x))

#Temas

In [ ]:
##################### GENERACION DE TABLA TEMAS
#Extracción del diccionario
# 2022-04-01: Ultima version que deberia devolver el dataframe:
# DicTemas = pd.read_excel('/dbfs/Efecty_VoC/BETOmodel/Diccionario temas.xlsx')
query = "SELECT ORIGEN, [SUBTIPO/WRAPUP/ENCUESTA], [TEMÁTICA] FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_TEMAS"
DicTemas = spark.read \
                 .format("jdbc") \
                 .option("url", synapse_url) \
                 .option("user", synapse_user) \
                 .option("password", synapse_password)\
                 .option("query", query) \
                 .load() \
                 .toPandas() 


#GENERO EL DIC DE TEMA Y TEMA_ID
# 2022-04-01: Ultima version que deberia devolver el dataframe:
# DicTemas2 = pd.read_excel('/dbfs/Efecty_VoC/BETOmodel/Diccionario temas.xlsx',sheet_name='Hoja2')
query = "SELECT TEMA_ID, TEMA FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_LK_TEMA"
DicTemas2 = spark.read \
                 .format("jdbc") \
                 .option("url", synapse_url) \
                 .option("user", synapse_user) \
                 .option("password", synapse_password)\
                 .option("query", query) \
                 .load() \
                 .orderBy('TEMA_ID') \
                 .toPandas() 

dic_tema_id=dict(list(zip(DicTemas2['TEMA'],DicTemas2['TEMA_ID'])))

DicTemas['TEMA_ID']=DicTemas['TEMÁTICA'].apply(lambda x: int(dic_tema_id[x]))
Dic_Temas_subtipo=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='Subtipo-PQRS'],
                                DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='Subtipo-PQRS'])))
Dic_Temas_wrapups=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='wrapups'],
                                DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='wrapups'])))
Dic_Temas_enc=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='Encuestas'],
                            DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='Encuestas'])))


BT_COMENTARIOS.reset_index(inplace=True)


#Paso a formato string las columnas donde voy a aplicar el diccionario
BT_COMENTARIOS['subtipo']=BT_COMENTARIOS['subtipo'].astype(str)
index_st=BT_COMENTARIOS[BT_COMENTARIOS['subtipo']=='None'].index
BT_COMENTARIOS.loc[index_st,'subtipo']=''
BT_COMENTARIOS['wrapup']=BT_COMENTARIOS['wrapup'].astype(str)
index_wr=BT_COMENTARIOS[BT_COMENTARIOS['wrapup']=='None'].index
BT_COMENTARIOS.loc[index_wr,'wrapup']=''
BT_COMENTARIOS['encuesta']=BT_COMENTARIOS['encuesta'].astype(str)
index_enc=BT_COMENTARIOS[BT_COMENTARIOS['encuesta']=='None'].index
BT_COMENTARIOS.loc[index_enc,'encuesta']=''

#Los index! se aplicará tema a todo lo que tenga o: subtipo, wrapup o encuesta
#Extracción de index para según origen
index_sub=BT_COMENTARIOS[BT_COMENTARIOS['subtipo']!=''].index
index_rrss=BT_COMENTARIOS[BT_COMENTARIOS['wrapup']!=''].index
index_enc=BT_COMENTARIOS[BT_COMENTARIOS['encuesta']!=''].index

#Alpico los diccionarios según los index
BT_COMENTARIOS.loc[index_enc,'tema_id']=BT_COMENTARIOS['encuesta'].apply(lambda x: Dic_Temas_enc.get(x) if  Dic_Temas_enc.get(x)!='' else x) 
BT_COMENTARIOS.loc[index_sub,'tema_id']=BT_COMENTARIOS['subtipo'].apply(lambda x: Dic_Temas_subtipo.get(x) if  Dic_Temas_subtipo.get(x)!='' else x) 
BT_COMENTARIOS.loc[index_rrss,'tema_id']=BT_COMENTARIOS['wrapup'].apply(lambda x: Dic_Temas_wrapups.get(x) if  Dic_Temas_wrapups.get(x)!='' else x) 


# Todo lo que tenga wrapup nulo y sea RRSS, debería tener temática Otros!!!

####### a todas las encuestas con sentimiento(indicador_id=4), les correra este modelo. Si no es el valor espedaro, hardcodear "Medición" o que efecty genere otro dataset para esa encuesta puntual
index_tem=BT_COMENTARIOS[(BT_COMENTARIOS['encuesta']!='')&(BT_COMENTARIOS['indicador_id']==4)].index

#Cargo el modelo
model = torch.load('/dbfs/Efecty_VoC/BETOmodel/modelV_categv3_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

#Asi se entrenó a modelV_categv3_BETO_uncased_fine_tune. no se pueden modificar

#Diccionario viejo
dic_PAPs={1:'Capacitación',2:'Convenios y franquicias',3:'Publicidad',
        4:'Otros',5:'Satisfacción',6:'Sieweb Live',0:'Soporte a PAPs',7:'Sin comentarios',8: 'Recaudo'}

#Diccionario nuevo, cuando metamos el nuevo modelo
# dic_PAPs={1:'Capacitación',2:'convenios gral',3:'Otros',
#         4:'Otros',5:'Otros',6:'Solicitud Sistema PAP',0:'PAPs',7:'Otros',8: 'Topes'}

#A cada tema, le asigno el id universal de temas, proveniente de "Diccionario temas.xlsx"
BT_COMENTARIOS.loc[index_tem,'tema_id']=BT_COMENTARIOS['comentario'].apply(lambda x:dic_tema_id[dic_PAPs[classifySentiment(x)]])

#Modifico el formato
BT_COMENTARIOS['tema_id']=BT_COMENTARIOS['tema_id'].apply(lambda x: -99 if pd.isna(x) else x)
BT_COMENTARIOS['tema_id']=BT_COMENTARIOS['tema_id'].apply(lambda x: int(x))


# ############# DESDE ACA, APLICO BETO_TEMAS PARA LOS NUEVOS TEMAS NO MAPEADOS INICIALMENTE

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/modelV_temasv5_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

#Genero los index para identificar los subtipos/wrapups/encuestas no mapeadas inicialmente 
index_tem_w=BT_COMENTARIOS[(BT_COMENTARIOS['encuesta']=='')&(BT_COMENTARIOS['subtipo']=='')&(BT_COMENTARIOS['tema_id']==-99)].index

if list(index_tem_w)!=[]:
    #Preparo el formato tal cual como se entreno
    BT_COMENTARIOS['wrapup2']=BT_COMENTARIOS['wrapup']
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x: str(x).lower())
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub(' dev.','devolución',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('inf.|inf ','información',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('/|Corresponsal Bancario -|1-|1|cliente corporativo|cliente natural|cliente|corporativo|usuario final|colaborado de red|P_|Q_|In_|S_',' ',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('\(|\)|_|-',' ',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('tu cuenta','addidas',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('   ',' ',x))
    BT_COMENTARIOS.loc[index_tem_w,'wrapup2']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x:   re.sub('  ',' ',x))
    #Aplico beto_temas a los WEC no mapeados
    BT_COMENTARIOS.loc[index_tem_w,'tema_id']=BT_COMENTARIOS.loc[index_tem_w,'wrapup2'].apply(lambda x: classifySentiment(x))


# #REPITO PROCESO PARA SUBTIPO
index_tem_s=BT_COMENTARIOS[(BT_COMENTARIOS['wrapup']=='')&(BT_COMENTARIOS['encuesta']=='')&(BT_COMENTARIOS['tema_id']==-99)].index
if list(index_tem_s)!=[]:
    #Preparo el formato tal cual como se entreno
    BT_COMENTARIOS['subtipo2']=BT_COMENTARIOS['subtipo']
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x: str(x).lower())
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub(' dev.','devolución',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('inf.|inf ','información',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('/|Corresponsal Bancario -|1-|1|cliente corporativo|cliente natural|cliente|corporativo|usuario final|colaborado de red|P_|Q_|In_|S_',' ',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('\(|\)|_|-',' ',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('tu cuenta','addidas',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('   ',' ',x))
    BT_COMENTARIOS.loc[index_tem_s,'subtipo2']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x:   re.sub('  ',' ',x))
    #Aplico beto_temas a los WEC no mapeados
    BT_COMENTARIOS.loc[index_tem_s,'tema_id']=BT_COMENTARIOS.loc[index_tem_s,'subtipo2'].apply(lambda x: classifySentiment(x))


#REPITO PROCESO PARA ENCUESTA
index_tem_e=BT_COMENTARIOS[(BT_COMENTARIOS['wrapup']=='')&(BT_COMENTARIOS['subtipo']=='')&(BT_COMENTARIOS['tema_id']==-99)].index
if list(index_tem_s)!=[]:
    #Preparo el formato tal cual como se entreno
    BT_COMENTARIOS['encuesta2']=BT_COMENTARIOS['encuesta']
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x: str(x).lower())
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub(' dev.','devolución',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('inf.|inf ','información',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('/|Corresponsal Bancario -|1-|1|cliente corporativo|cliente natural|cliente|corporativo|usuario final|colaborado de red|P_|Q_|In_|S_',' ',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('\(|\)|_|-',' ',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('tu cuenta','addidas',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('   ',' ',x))
    BT_COMENTARIOS.loc[index_tem_e,'encuesta2']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x:   re.sub('  ',' ',x))
    #Aplico beto_temas a los WEC no mapeados
    BT_COMENTARIOS.loc[index_tem_e,'tema_id']=BT_COMENTARIOS.loc[index_tem_e,'encuesta2'].apply(lambda x: classifySentiment(x))
    

#Genero la tabla diccionario
schema = StructType([
    StructField('TEMA_ID', IntegerType(), False),
    StructField('TEMA', StringType(), False)
])

dic_tema_id_invertido = dict(list(zip(DicTemas2['TEMA_ID'],DicTemas2['TEMA'])))
LK_TEMA = spark.createDataFrame(dic_tema_id_invertido.items(), schema) 
nombre_archivo = f'{path_contenedor_lk}/LK_TEMA'
LK_TEMA.write.mode('overwrite').parquet(nombre_archivo)

#Frases / Causa Raiz

In [ ]:
############################# EXTRACCION DE FRASES/CAUSA RAIZ ######################################

#Se entreno solo para encuestas CRM,indicador_id=4

index_CR=BT_COMENTARIOS[(BT_COMENTARIOS['canal']=='PQRS')|(BT_COMENTARIOS['canal']=='RRSS')|
                ((~BT_COMENTARIOS['encuesta'].isna())&(BT_COMENTARIOS['indicador_id']==4)&(BT_COMENTARIOS['medio_ingreso']=='CRM'))].index

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/modelV_causaraizv1_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_causa_raiz={0:'Producto',    
#         1:'Operativo',
#         2:'Mantenimiento',
#         3:'Trato con cliente',
#         4:'Precio',
#         5:'No Aplica'
#         }
query = "SELECT CAUSA_RAIZ_ID, CAUSA_RAIZ FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_LK_CAUSA_RAIZ"
dict_causa_raiz = spark.read \
                        .format("jdbc") \
                        .option("url", synapse_url) \
                        .option("user", synapse_user) \
                        .option("password", synapse_password)\
                        .option("query", query) \
                        .load() \
                        .orderBy('CAUSA_RAIZ_ID') \
                        .toPandas() \
                        .set_index('CAUSA_RAIZ_ID') \
                        .T \
                        .to_dict('index')['CAUSA_RAIZ']


BT_COMENTARIOS.loc[index_CR,'comentario_CR']=BT_COMENTARIOS.loc[index_CR,'comentario'].apply(lambda x:re.sub('[0-9]{1,20}','numero',x))
BT_COMENTARIOS.loc[index_CR,'comentario_CR']=BT_COMENTARIOS.loc[index_CR,'comentario_CR'].apply(lambda x:re.sub('\W',' ',x))
BT_COMENTARIOS.loc[index_CR,'causa_raiz_id']=BT_COMENTARIOS.loc[index_CR,'comentario_CR'].apply(lambda x:classifySentiment(x))


#Genero la tabla diccionario
schema = StructType([
    StructField('CAUSA_RAIZ_ID', IntegerType(), False),
    StructField('CAUSA_RAIZ', StringType(), False)
])

LK_CAUSA_RAIZ = spark.createDataFrame(dict_causa_raiz.items(), schema) 
nombre_archivo = f'{path_contenedor_lk}/LK_CAUSA_RAIZ'
LK_CAUSA_RAIZ.write.mode('overwrite').parquet(nombre_archivo)

#Palabra

In [ ]:
############################# EXTRACCION DE PALABRA ######################################
# descargar 'es_core_news_lg' (python -m spacy download es_core_news_lg)
# subprocess.run(python -m spacy download es_core_news_lg)

nlp = spacy.load('es_core_news_lg')

BT_COMENTARIOS['palabra']=''


index_P=BT_COMENTARIOS[(BT_COMENTARIOS['medio_ingreso']!='LLAMADA')&(BT_COMENTARIOS['medio_ingreso']!='IVR')&((BT_COMENTARIOS['indicador_id']!=1)&(BT_COMENTARIOS['indicador_id']!=2)&(BT_COMENTARIOS['indicador_id']!=3))].index

# for i in range(0,len(BT_COMENTARIOS['comentario'])):
for i in index_P:
    try:
        doc=nlp(BT_COMENTARIOS['comentario'][i])
        g1=[]
        for  token in doc:
            if (token.pos_=='NOUN')&(token.text!='número'):
                g1.append(token.text)
        BT_COMENTARIOS['palabra'][i]=g1
    except:
        pass

BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: ', '.join(x))
BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: re.sub("gracias|efecty|favor|cliente|día|dias|usuario|referencia|correo|usuaria|dia|dias|buen|buenos|numero|número|saludos|saludo|señores|caso",'',x))
BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: re.sub("[0-9]{1,20}",'',x))

BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: re.sub(r'\\n',' ',x))
BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: re.sub('\W',' ',x))
BT_COMENTARIOS['palabra']=BT_COMENTARIOS['palabra'].apply(lambda x: re.sub('  ',' ',x))

#Punto Interaccion

In [ ]:
############################# EXTRACCION DE PUNTO INTERACCIÓN ######################################

#Se entreno solo para encuestas CRM,indicador_id=4

index_PI_Encuesta=BT_COMENTARIOS[(((BT_COMENTARIOS['tipo_cliente']!='NO IDENTIFICADO')&(BT_COMENTARIOS['tipo_cliente']!='NATURAL'))&(BT_COMENTARIOS['tipo_cliente']!='USUARIO FINAL'))
                                 &(((BT_COMENTARIOS['encuesta']!='')&(BT_COMENTARIOS['indicador_id']==4)&(BT_COMENTARIOS['medio_ingreso']!='IVR'))|(BT_COMENTARIOS['canal']=='PQRS')|(BT_COMENTARIOS['canal']=='RRSS')| (BT_COMENTARIOS['canal']=='CUSTOMER SERVICE'))].index

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/model_Punto_interaccionV1_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

#diccionarios del modelo, inmodificables
# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_PI_inverse={ 0:'Experiencia del Cliente',
# 1:'Facilitador Comercial',
# 2:'Facilitador de Canales',
# 3:'Facilitador de Negocios',
# 4:'Gestión Interna',
# 5:'Mesa de servicio',
# 6:'No mapeado',
# 7:'Operaciones (Regional)',
# 9:'PAP',
# 10:'Portal',
# 8:'Sistema (Sieweb live)',
# 11:'Canal Virtual'} #Este 11 (canalvirtual) esta creado, super hardcodeado. cuando tengamos un modelo de este para usuario final, elimiarlo

query = "SELECT PUNTO_INTERACCION_ID, PUNTO_INTERACCION, CN_UF, PREGUNTA FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_LK_PUNTO_INTERACCION"
dict_spark = spark.read \
                 .format("jdbc") \
                 .option("url", synapse_url) \
                 .option("user", synapse_user) \
                 .option("password", synapse_password)\
                 .option("query", query) \
                 .load() 

dict_PI_inverse = dict_spark.select('PUNTO_INTERACCION_ID','PUNTO_INTERACCION').orderBy('PUNTO_INTERACCION_ID').toPandas().set_index('PUNTO_INTERACCION_ID').T.to_dict('index')['PUNTO_INTERACCION']

BT_COMENTARIOS['punto_interaccion_id']=-99
BT_COMENTARIOS.loc[index_PI_Encuesta,'punto_interaccion_id']=BT_COMENTARIOS.loc[index_PI_Encuesta,'comentario'].apply(lambda x:classifySentiment(x))


############ DESDE ACA, CORRER UN MODELO PARA NATURAL Y USUARIO FINAL (cuando se haga, eliminar el 11 de arriba)

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/PI para cliente final e naturalV2.pkl', map_location=torch.device('cpu'))

index_PI_UFyCN=BT_COMENTARIOS[(((BT_COMENTARIOS['tipo_cliente']=='NO IDENTIFICADO')|(BT_COMENTARIOS['tipo_cliente']=='NATURAL'))|(BT_COMENTARIOS['tipo_cliente']=='USUARIO FINAL'))
                                 &(((BT_COMENTARIOS['encuesta']!='')&(BT_COMENTARIOS['indicador_id']==4)&(BT_COMENTARIOS['medio_ingreso']!='IVR'))|(BT_COMENTARIOS['canal']=='PQRS')|(BT_COMENTARIOS['canal']=='RRSS')| (BT_COMENTARIOS['canal']=='CUSTOMER SERVICE'))].index

# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_PI_CNyUF={0: 0,
#  1: 11,
#  2: 12,
#  3: 6,
#  4: 4,
#  5: 9}

dict_PI_CNyUF = dict_spark.filter('CN_UF IS NOT NULL').select('CN_UF','PUNTO_INTERACCION_ID').orderBy('CN_UF').toPandas().set_index('CN_UF').T.to_dict('index')['PUNTO_INTERACCION_ID']

BT_COMENTARIOS.loc[index_PI_UFyCN,'punto_interaccion_id']=BT_COMENTARIOS.loc[index_PI_UFyCN,'comentario'].apply(lambda x:dict_PI_CNyUF.get(classifySentiment(x)))


##### punto de interaccion a preguntas encuestas

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/model_Punto_interaccion_preguntas_v1_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_PI_preg__={0:11,
#  1:12,
#  2:0,
#  3:1,
#  4:2,
#  5:3,
#  6:4,
#  7:5,
#  8:6,
#  9:7,
#  10:9,
#  11:10}

dict_PI_preg__ = dict_spark.filter('PREGUNTA IS NOT NULL').select('PREGUNTA','PUNTO_INTERACCION_ID').orderBy('PREGUNTA').toPandas().set_index('PREGUNTA').T.to_dict('index')['PUNTO_INTERACCION_ID']


index_Preg_Encuesta=BT_COMENTARIOS[(BT_COMENTARIOS['canal']=='ENCUESTA')&(BT_COMENTARIOS['indicador_id']!=4)].index

BT_COMENTARIOS.loc[index_Preg_Encuesta,'punto_interaccion_id']=BT_COMENTARIOS.loc[index_Preg_Encuesta,'pregunta'].apply(lambda x:dict_PI_preg__.get(classifySentiment(x)))

#Genero la tabla diccionario
schema = StructType([
    StructField('PUNTO_INTERACCION_ID', IntegerType(), False),
    StructField('PUNTO_INTERACCION', StringType(), False)
])

LK_PUNTO_INTERACCION = spark.createDataFrame(dict_PI_inverse.items(), schema) 
nombre_archivo = f'{path_contenedor_lk}/LK_PUNTO_INTERACCION'
LK_PUNTO_INTERACCION.write.mode('overwrite').parquet(nombre_archivo)

#Customer Journey

In [ ]:
############################# EXTRACCION DE CUSTOMER JOURNEY - NO NATURAL/FINAL ######################################
#Se entreno solo para encuestas CRM,indicador_id=4

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/model_CustomerJourneyV1_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

BT_COMENTARIOS['cliente']=BT_COMENTARIOS['encuesta'].apply(lambda x: ''.join(re.findall('(.*) -',x)))
BT_COMENTARIOS['punto_interaccion']=BT_COMENTARIOS['punto_interaccion_id'].apply(lambda x: dict_PI_inverse.get(x))
BT_COMENTARIOS['encuesta3']=BT_COMENTARIOS['encuesta'].apply(lambda x: ''.join(re.findall('-(.*)',x)))
BT_COMENTARIOS['encuesta3']=BT_COMENTARIOS['encuesta3'].apply(lambda x: re.sub('\dM\d\d','satisfacción',x))

BT_COMENTARIOS['COMENTARIO_CJ']='El '+BT_COMENTARIOS['cliente']+' hablando de '+BT_COMENTARIOS['punto_interaccion']+' dijo: '+BT_COMENTARIOS['comentario']

BT_COMENTARIOS['customer_journey_id']=-99

index_CJ_g1=BT_COMENTARIOS[(
    ((BT_COMENTARIOS['tipo_cliente']!='NATURAL')|(BT_COMENTARIOS['tipo_cliente']!='USUARIO FINAL'))& 
    (
        ((~BT_COMENTARIOS['encuesta'].isna())&(BT_COMENTARIOS['indicador_id']==4)|(BT_COMENTARIOS['medio_ingreso']!='IVR'))|
        (BT_COMENTARIOS['canal']=='PQRS')|
        (BT_COMENTARIOS['canal']=='RRSS')|
        (BT_COMENTARIOS['canal']=='CUSTOMER SERVICE')))&(~BT_COMENTARIOS['COMENTARIO_CJ'].isna())].index


BT_COMENTARIOS.loc[index_CJ_g1,'customer_journey_id']=BT_COMENTARIOS.loc[index_CJ_g1,'COMENTARIO_CJ'].apply(lambda x:classifySentiment(x))


########################## 21/3/2022 CJ COLAB Y CORPO ENCUESTAS
##SOLO PARA ENCUESTAS
BT_COMENTARIOS['COMENTARIO_CJ']='El '+BT_COMENTARIOS['cliente']+' en la encuesta '+BT_COMENTARIOS['encuesta3']+' le preguntaron, '+BT_COMENTARIOS['pregunta']+' hablando de '+BT_COMENTARIOS['punto_interaccion']+' dijo: '+BT_COMENTARIOS['comentario']


index_CJ_g1_En=BT_COMENTARIOS[(
    ((BT_COMENTARIOS['tipo_cliente']!='NATURAL')|(BT_COMENTARIOS['tipo_cliente']!='USUARIO FINAL'))& 
        (BT_COMENTARIOS['indicador_id']==4)&(BT_COMENTARIOS['medio_ingreso']!='IVR')&(BT_COMENTARIOS['canal']=='ENCUESTA')&(~BT_COMENTARIOS['COMENTARIO_CJ'].isna()))].index

BT_COMENTARIOS.loc[index_CJ_g1_En,'customer_journey_id']=BT_COMENTARIOS.loc[index_CJ_g1_En,'COMENTARIO_CJ'].apply(lambda x:classifySentiment(x))

# Hardcodeado. sucede que no tenemos suficientes muestras para hacerlo correctamente. en el dataset que nos pasaron, todo lo que es 'Colaborador de la Red - 2M21' con 2 preguntas de mesa de servicio y auxilar, son de 'Soporte continuo' y hay respuestas de SI y NO en su mayoría.
index_1CJ=BT_COMENTARIOS[((BT_COMENTARIOS['comentario']=='SI')|(BT_COMENTARIOS['comentario']=='NO'))&(BT_COMENTARIOS['encuesta']=='Colaborador de la Red - 2M21')].index
BT_COMENTARIOS.loc[index_1CJ,'customer_journey_id']=0

In [ ]:
############################# EXTRACCION DE CUSTOMER JOURNEY - CLIENTE NATURAL/FINAL/NO IDENTIFICADO ######################################
model = torch.load('/dbfs/Efecty_VoC/BETOmodel/CJ para cliente final e naturalV6.pkl', map_location=torch.device('cpu'))


#Los ids para los clientes naturales y usuario final. Los agregue a mano en la tabla LK_CUSTOMER_JOURNEY
# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_CJ_preg={5: 'Alistamiento',
#  51: 'Atención al Cliente',
#  52: 'Compra',
#  8: 'Desvinculación',
#  1: 'Mantenimiento comercial de la cuenta',
#  3: 'Mantenimiento del PAP',
#  56: 'No Mapeado',
#  9: 'Parametrización del proyecto',
#  7: 'Percepción de Marca',
#  59: 'Precompra',
#  2: 'Previnculación',
#  6: 'Salida a Producción',
#  0: 'Servicio y soporte continuo',
#  4: 'Vinculación'}

query = "SELECT CUSTOMER_JOURNEY_ID, CUSTOMER_JOURNEY, CN_UF, PREGUNTA FROM EFECTY_VOC_MODEL_NLP_LANDING.PARAM_LK_CUSTOMER_JOURNEY"
dict_spark = spark.read \
                 .format("jdbc") \
                 .option("url", synapse_url) \
                 .option("user", synapse_user) \
                 .option("password", synapse_password)\
                 .option("query", query) \
                 .load() 

dict_CJ_preg = dict_spark.select('CUSTOMER_JOURNEY_ID','CUSTOMER_JOURNEY').orderBy('CUSTOMER_JOURNEY_ID').toPandas().set_index('CUSTOMER_JOURNEY_ID').T.to_dict('index')['CUSTOMER_JOURNEY']


index_CJ_g2=BT_COMENTARIOS[(
    ((BT_COMENTARIOS['tipo_cliente']=='NATURAL')|(BT_COMENTARIOS['tipo_cliente']=='USUARIO FINAL')|(BT_COMENTARIOS['tipo_cliente']=='NO IDENTIFICADO'))& 
    (
        ((~BT_COMENTARIOS['encuesta'].isna())&(BT_COMENTARIOS['indicador_id']==4)|(BT_COMENTARIOS['medio_ingreso']!='IVR'))|
        (BT_COMENTARIOS['canal']=='PQRS')|
        (BT_COMENTARIOS['canal']=='RRSS')|
        (BT_COMENTARIOS['canal']=='CUSTOMER SERVICE')))&(~BT_COMENTARIOS['COMENTARIO_CJ'].isna())].index



#Modelo que corre para CF y CN (revisar, con maru para ampliar el margen)
# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_CJ_CNyF={0:59,
#  1:52,
# 2:7,
# 3:51,
# 4:56}

dict_CJ_CNyF = dict_spark.filter('CN_UF IS NOT NULL').select('CN_UF','CUSTOMER_JOURNEY_ID').orderBy('CN_UF').toPandas().set_index('CN_UF').T.to_dict('index')['CUSTOMER_JOURNEY_ID']

BT_COMENTARIOS.loc[index_CJ_g2,'customer_journey_id']=BT_COMENTARIOS.loc[index_CJ_g2,'comentario'].apply(lambda x:dict_CJ_CNyF.get(classifySentiment(x)))


####APLICO CJ A LAS PREGUNTAS DE LAS ENCUESTAS -- NATURAL Y NO NATURAL

model = torch.load('/dbfs/Efecty_VoC/BETOmodel/model_CustomerJourney_preguntas_v1_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

index_Preg_Encuesta=BT_COMENTARIOS[(BT_COMENTARIOS['canal']=='ENCUESTA')&(BT_COMENTARIOS['indicador_id']!=4)].index

# 2022-04-01: Ultima version que deberia devolver el diccionario:
# dict_CJ_preg__={0:5,
#  1:51,
#  2:52,
#  3:8,
#  4:1,
#  5:3,
#  6:56,
#  7:9,
#  8:7,
#  9:59,
#  10:2,
#  11:6,
#  12:0,
#  13:4}

dict_CJ_preg__ = dict_spark.filter('PREGUNTA IS NOT NULL').select('PREGUNTA','CUSTOMER_JOURNEY_ID').orderBy('PREGUNTA').toPandas().set_index('PREGUNTA').T.to_dict('index')['CUSTOMER_JOURNEY_ID']

BT_COMENTARIOS.loc[index_Preg_Encuesta,'customer_journey_id']=BT_COMENTARIOS.loc[index_Preg_Encuesta,'pregunta'].apply(lambda x:dict_CJ_preg__.get(classifySentiment(x)))

#Genero la tabla diccionario
schema = StructType([
    StructField('CUSTOMER_JOURNEY_ID', IntegerType(), False),
    StructField('CUSTOMER_JOURNEY', StringType(), False)
])

LK_CUSTOMER_JOURNEY = spark.createDataFrame(dict_CJ_preg.items(), schema) 
nombre_archivo = f'{path_contenedor_lk}/LK_CUSTOMER_JOURNEY'
LK_CUSTOMER_JOURNEY.write.mode('overwrite').parquet(nombre_archivo)

#Armado Tabla Final

In [ ]:
######################## PREPROCESO DE TABLAS  #########################

BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario_inicial']
df_bt_comentarios=BT_COMENTARIOS.copy()

# columnas de bt_comentarios
df_bt_comentarios=df_bt_comentarios[['id','interaccion_id','pregunta_id','comentario','wrapupnote','wrapup_id','valor','subtipo_bt','fecha_proceso','clase_id','dato_id','medio_ingreso_id','sentimiento','tema_id','indicador_id','causa_raiz_id','palabra','punto_interaccion_id','customer_journey_id']] 
df_bt_comentarios=df_bt_comentarios.rename(columns={'subtipo_bt':'subtipo'})

In [ ]:
# Doy formato a las columnas
df_bt_comentarios['comentario'].fillna('',inplace=True)
df_bt_comentarios['comentario']=df_bt_comentarios['comentario'].astype(pd.StringDtype())
df_bt_comentarios['wrapupnote'].fillna('',inplace=True)
df_bt_comentarios['wrapupnote']=df_bt_comentarios['wrapupnote'].astype(pd.StringDtype())
df_bt_comentarios['palabra'].fillna('',inplace=True)
df_bt_comentarios['palabra']=df_bt_comentarios['palabra'].astype(pd.StringDtype())
df_bt_comentarios['dato_id'].fillna('',inplace=True)
df_bt_comentarios['dato_id']=df_bt_comentarios['dato_id'].astype(pd.StringDtype())

df_bt_comentarios['medio_ingreso_id'].fillna(0,inplace=True)
df_bt_comentarios['medio_ingreso_id']=df_bt_comentarios['medio_ingreso_id'].astype(int)
df_bt_comentarios['pregunta_id'].fillna(0,inplace=True)
df_bt_comentarios['pregunta_id']=df_bt_comentarios['pregunta_id'].astype(int)
df_bt_comentarios['wrapup_id'].fillna(0,inplace=True)
df_bt_comentarios['wrapup_id']=df_bt_comentarios['wrapup_id'].astype(int)
df_bt_comentarios['clase_id'].fillna(0,inplace=True)
df_bt_comentarios['clase_id']=df_bt_comentarios['clase_id'].astype(int)
df_bt_comentarios['valor'].fillna(0,inplace=True)
df_bt_comentarios['valor']=df_bt_comentarios['valor'].astype(int)
df_bt_comentarios['subtipo'].fillna(0,inplace=True)
df_bt_comentarios['subtipo']=df_bt_comentarios['subtipo'].astype(int)

df_bt_comentarios.fillna(-99,inplace=True)
df_bt_comentarios['sentimiento']=df_bt_comentarios['sentimiento'].astype(float)
df_bt_comentarios['interaccion_id']=df_bt_comentarios['interaccion_id'].astype(int)
df_bt_comentarios['indicador_id']=df_bt_comentarios['indicador_id'].astype(int)
df_bt_comentarios['tema_id']=df_bt_comentarios['tema_id'].astype(int)
df_bt_comentarios['interaccion_id']=df_bt_comentarios['interaccion_id'].astype(int)
df_bt_comentarios['punto_interaccion_id']=df_bt_comentarios['punto_interaccion_id'].astype(int)
df_bt_comentarios['customer_journey_id']=df_bt_comentarios['customer_journey_id'].astype(int)


# df_bt_comentarios.replace(-99,np.NaN,inplace=True)

In [ ]:
#Paso a formato upper
listr=[]
for i in df_bt_comentarios.columns.tolist():
    listr.append(i.upper())
dict_upper=dict(zip(list(df_bt_comentarios.columns),listr))
df_bt_comentarios=df_bt_comentarios.rename(columns=dict_upper)


df_bt_comentarios=df_bt_comentarios[['MEDIO_INGRESO_ID','DATO_ID','ID', 'INTERACCION_ID', 'PREGUNTA_ID','TEMA_ID', 'COMENTARIO', 'WRAPUPNOTE',
       'WRAPUP_ID', 'VALOR', 'SUBTIPO', 'SENTIMIENTO','FECHA_PROCESO', 'CLASE_ID','INDICADOR_ID', 'CAUSA_RAIZ_ID', 
        'PALABRA','PUNTO_INTERACCION_ID', 'CUSTOMER_JOURNEY_ID']]

#Generacion de Parquet

In [ ]:
nombre_archivo = f'{path_contenedor_bt}/Corrida-BT_COMENTARIOS_{var_FechaDesde}_{var_FechaHasta}'
py_insert = spark.createDataFrame(df_bt_comentarios)
py_insert.write.mode('overwrite').parquet(nombre_archivo)